In [7]:
from data import *
from models import *
from utils import *

from sklearn.metrics import f1_score, accuracy_score

MAE_DIR = '/home/neelamlab/ninad/MAE'
ROOT_DIR = '/data/ninad/DATASET_linear'
LABEL_DIR = f'/data/ninad/Metadata'
RESULT_DIR = f'{MAE_DIR}/results'
LOG_DIR = f'{MAE_DIR}/logs/'
TASK = 'regression'
CDRGLOB = 0.0
ONLY_3T_SCANS = 0.001

NP_SEED = 42
TORCH_SEED = 36
np.random.seed(NP_SEED)
torch.manual_seed(TORCH_SEED)

LOGGER = setup_logger(logs_dir=LOG_DIR)

In [4]:
RESULT_DIR = 'results'
filename = 'test_predictions_TEST_DATA.csv'
runmetrics = {}
age_threshold = 0
MIN_SEED = 11
MAX_SEED = 15
SKIP_SEED = 20
for run in os.listdir(RESULT_DIR):
    if run[:4]=='SFCN' and MIN_SEED<=int(run.split('_')[-1])<=MAX_SEED and int(run.split('_')[-1])!=SKIP_SEED:
        print(run)
        for fold in range(5):
            print(os.path.join(RESULT_DIR, run, f'fold_{fold}',filename))
            # runmetrics[run] = {}
            if os.path.exists(os.path.join(RESULT_DIR, run, f'fold_{fold}',filename)):
                true, predicted = [],[]
                for dataset_name, _, i,j in pd.read_csv(os.path.join(RESULT_DIR, run, f'fold_{fold}', filename)).values:
                    if True: #dataset_name == 'CBR': #dataset_name != 'CBR' and dataset_name != 'LASI': #dataset_name == 'CBR': #
                        if i < age_threshold:
                            continue
                        true.append(i)
                        predicted.append(j)
                # plot_brain_age_delta(true, predicted, name=f'{run} TEST')
                runmetrics[f'{run}_f{fold}'] = {}
                runmetrics[f'{run}_f{fold}']['F1'] = f1_score(true,predicted)
                runmetrics[f'{run}_f{fold}']['Acc'] = accuracy_score(true,predicted)

SFCN_seed_12
results/SFCN_seed_12/fold_0/test_predictions_TEST_DATA.csv
results/SFCN_seed_12/fold_1/test_predictions_TEST_DATA.csv
results/SFCN_seed_12/fold_2/test_predictions_TEST_DATA.csv
results/SFCN_seed_12/fold_3/test_predictions_TEST_DATA.csv
results/SFCN_seed_12/fold_4/test_predictions_TEST_DATA.csv
SFCN_seed_15
results/SFCN_seed_15/fold_0/test_predictions_TEST_DATA.csv
results/SFCN_seed_15/fold_1/test_predictions_TEST_DATA.csv
results/SFCN_seed_15/fold_2/test_predictions_TEST_DATA.csv
results/SFCN_seed_15/fold_3/test_predictions_TEST_DATA.csv
results/SFCN_seed_15/fold_4/test_predictions_TEST_DATA.csv
SFCN_seed_11
results/SFCN_seed_11/fold_0/test_predictions_TEST_DATA.csv
results/SFCN_seed_11/fold_1/test_predictions_TEST_DATA.csv
results/SFCN_seed_11/fold_2/test_predictions_TEST_DATA.csv
results/SFCN_seed_11/fold_3/test_predictions_TEST_DATA.csv
results/SFCN_seed_11/fold_4/test_predictions_TEST_DATA.csv
SFCN_seed_14
results/SFCN_seed_14/fold_0/test_predictions_TEST_DATA.csv
resu

In [5]:
df = pd.DataFrame.from_dict(runmetrics).T
mean_values = df.mean().round(4)
std_values = df.std().round(4)
mean_std = mean_values.astype(str) + " ± " + std_values.astype(str)
# df.loc['mean ± std'] = mean_std
df.head()

,F1,Acc
SFCN_seed_12_f0,0.945813,0.937143
SFCN_seed_12_f1,0.948454,0.942857
SFCN_seed_12_f2,0.943005,0.937143
SFCN_seed_12_f3,0.942408,0.937143
SFCN_seed_12_f4,0.940594,0.931429


In [6]:
df['Seed'] = df.index.to_series().apply(lambda x: '_'.join(x.split('_')[:3]))
summary_df = df.groupby('Seed').agg({'F1': ['mean', 'std'], 'Acc': ['mean', 'std']})
summary_df.columns = ['_'.join(col) for col in summary_df.columns]
print(summary_df)

               F1_mean    F1_std  Acc_mean   Acc_std
Seed                                                
SFCN_seed_11  0.927080  0.018121  0.923429  0.015959
SFCN_seed_12  0.944055  0.003091  0.937143  0.004041
SFCN_seed_13  0.942291  0.005407  0.934857  0.005111
SFCN_seed_14  0.907314  0.007806  0.890286  0.012388
SFCN_seed_15  0.915475  0.010108  0.902857  0.014569


In [8]:
import os
import nibabel as nib
import pandas as pd
import torch
from torch.utils.data import Dataset

class Fieldmapdata_4updated(Dataset):
    def __init__(self, root_dir, label_dir, task='classification'):
        self.labels_df = self.load_labels(label_dir)
        self.samples = self.make_dataset(root_dir, task=task)

    def __len__(self):
        return len(self.samples)
    
    def __getposweight__(self):
        sexs = []
        for paths, sex in self.samples:
            sexs.append(sex)
        return sum(sexs) / (len(sexs) - sum(sexs))

    def __getitem__(self, idx):
        img_paths, label, dataset, id_ = self.samples[idx]
        nifti_tensors = []
        for img_path in img_paths:
            nifti_data = nib.load(img_path)
            data = nifti_data.get_fdata()
            nifti_tensors.append(torch.tensor(data, dtype=torch.float32))
        
        image_tensor = torch.stack(nifti_tensors, dim=0)
        label_tensor = torch.tensor(label, dtype=torch.float32).unsqueeze(0)
        label_tensor = (label_tensor, dataset, id_)
        return image_tensor, label_tensor
       
    def make_dataset(self, root_dir, task='regression'):
        samples = []
        labels_df = self.labels_df

        # Define subdirectories corresponding to the four fieldmaps
        subdirs = ['AD', 'RD', 'ADC', 'FA']

        # Collect subject IDs and their corresponding fieldmap paths
        subject_files = {}
        for subdir in subdirs:
            subdir_path = os.path.join(root_dir, subdir)
            for fname in os.listdir(subdir_path):
                if fname.endswith(".nii.gz") or fname.endswith(".nii"):
                    id_ = self.extract_id_from_filename(fname)
                    path = os.path.join(subdir_path, fname)
                    if id_ not in subject_files:
                        subject_files[id_] = {}
                    subject_files[id_][subdir] = path

        # Ensure each subject has all four fieldmaps
        for id_, paths_dict in subject_files.items():
            if all(subdir in paths_dict for subdir in subdirs):  # Check if all four fieldmaps are present
                try:
                    paths = [paths_dict[subdir] for subdir in subdirs]  # Ordered list of paths
                    if task == 'regression': 
                        label = labels_df[labels_df['Subject'] == id_]['Age'].iloc[0]
                    else:
                        label = labels_df[labels_df['Subject'] == id_]['Gender'].iloc[0]
                    samples.append((paths, label, 'HCP', id_))
                except Exception:
                    continue
        return samples
    
    def extract_id_from_filename(self, fname):
        fname = fname.replace("sub-","")
        if fname.endswith("_ad.nii.gz"):
            id_ = fname.replace("_ad.nii.gz", "")
        elif fname.endswith("_rd.nii.gz"):
            id_ = fname.replace("_rd.nii.gz", "")
        elif fname.endswith("_adc.nii.gz"):
            id_ = fname.replace("_adc.nii.gz", "")
        elif fname.endswith("_fa.nii.gz"):
            id_ = fname.replace("_fa.nii.gz", "")
        elif fname.endswith(".nii.gz"):
            id_ = fname.replace(".nii.gz", "")
        return id_

    def load_labels(self, label_path):
        df = pd.read_csv(label_path)
        df_filtered = df[['Subject', 'Gender', 'Age']].copy()
        df_filtered['Gender'] = df_filtered['Gender'].map({'M': 0, 'F': 1}).astype(int)
        df_filtered['Age'] = df_filtered['Age'].apply(lambda x: (int(x.split('-')[0]) + int(x.split('-')[1])) // 2 if '-' in x else int(x[:-1])).astype(float)
        df_filtered['Subject'] = df_filtered['Subject'].astype(str)
        return df_filtered


In [10]:
a = Fieldmapdata_4updated(root_dir='/data/ninad/fieldmaps/',label_dir='/data/ninad/fieldmaps/HCPCN2mm.csv')
a.__len__()

850